In [79]:
# import libraries

import numpy as np
import string
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords

In [80]:
# read from the json file. Has a shitton of files but hey it works
card_df = pd.read_json(path_or_buf='oracle-cards-20231211100140.json', orient='records')


In [81]:
# simplify down to only the Important Features

IF_df = card_df[['oracle_id', 'name', 'type_line', 'keywords', 'oracle_text']]
IF_df

,oracle_id,name,type_line,keywords,oracle_text
0,0004ebd0-dfd6-4276-b4a6-de0003e94237,Static Orb,Artifact,[],"As long as Static Orb is untapped, players can..."
1,0006faf6-7a61-426c-9034-579f2cfcfa83,Sensory Deprivation,Enchantment — Aura,[Enchant],Enchant creature\nEnchanted creature gets -3/-0.
2,0007c283-5b7a-4c00-9ca1-b455c8dff8c3,Road of Return,Sorcery,[Entwine],Choose one —\n• Return target permanent card f...
3,000d5588-5a4c-434e-988d-396632ade42c,Storm Crow,Creature — Bird,[Flying],Flying (This creature can't be blocked except ...
4,000e5d65-96c3-498b-bd01-72b1a1991850,Walking Sponge,Creature — Sponge,[],{T}: Target creature loses your choice of flyi...
...,...,...,...,...,...
30448,fffa9334-3576-4f70-9605-2ad062cdbc69,Without Weakness,Instant,[Cycling],Target creature you control gains indestructib...
30449,fffaa634-42a0-4038-b027-24f28754fec7,Firesong and Sunspeaker,Legendary Creature — Minotaur Cleric,[],Red instant and sorcery spells you control hav...
30450,fffcb71f-e802-436e-917d-eaa6607ab74f,"Toralf, God of Fury // Toralf's Hammer",Legendary Creature — God // Legendary Artifact...,"[Trample, Equip]",NaN
30451,fffdc2ac-bde4-4e4c-a5bd-0e6c6e49ad91,"Samut, the Tested",Legendary Planeswalker — Samut,[],+1: Up to one target creature gains double str...


In [82]:
# split type line into supertype and subtype features
# then remove 'type_line'
# for cards like Toralf, I can see that this will cause problems, but it is not big enough of a problem to care rn


IF_df.loc[:,['super_type']] = IF_df['type_line'].apply(lambda x: x.split('—', 1)[0] if '—' in x else x)
IF_df.loc[:,['sub_type']] = IF_df['type_line'].apply(lambda x: x.split('—', 1)[1] if '—' in x else '')
IF_df = IF_df[['oracle_id', 'name', 'super_type', 'sub_type', 'keywords', 'oracle_text']]
IF_df

,oracle_id,name,super_type,sub_type,keywords,oracle_text
0,0004ebd0-dfd6-4276-b4a6-de0003e94237,Static Orb,Artifact,,[],"As long as Static Orb is untapped, players can..."
1,0006faf6-7a61-426c-9034-579f2cfcfa83,Sensory Deprivation,Enchantment,Aura,[Enchant],Enchant creature\nEnchanted creature gets -3/-0.
2,0007c283-5b7a-4c00-9ca1-b455c8dff8c3,Road of Return,Sorcery,,[Entwine],Choose one —\n• Return target permanent card f...
3,000d5588-5a4c-434e-988d-396632ade42c,Storm Crow,Creature,Bird,[Flying],Flying (This creature can't be blocked except ...
4,000e5d65-96c3-498b-bd01-72b1a1991850,Walking Sponge,Creature,Sponge,[],{T}: Target creature loses your choice of flyi...
...,...,...,...,...,...,...
30448,fffa9334-3576-4f70-9605-2ad062cdbc69,Without Weakness,Instant,,[Cycling],Target creature you control gains indestructib...
30449,fffaa634-42a0-4038-b027-24f28754fec7,Firesong and Sunspeaker,Legendary Creature,Minotaur Cleric,[],Red instant and sorcery spells you control hav...
30450,fffcb71f-e802-436e-917d-eaa6607ab74f,"Toralf, God of Fury // Toralf's Hammer",Legendary Creature,God // Legendary Artifact — Equipment,"[Trample, Equip]",NaN
30451,fffdc2ac-bde4-4e4c-a5bd-0e6c6e49ad91,"Samut, the Tested",Legendary Planeswalker,Samut,[],+1: Up to one target creature gains double str...


In [13]:
# This has tons of data we don't need for these purposes, so we'll remove... most of it
# 'language', 'layout', 
#card_df = card_df[['oracle_id', 'color_identity', 'colors', 'keywords', 'legalities', 'mana_cost', 'name', 'oracle_text', 'power', 'produced_mana', 'toughness', 'type_line']]
#print(card_df)

# actually im now realizing that I just want to split the dataframes up into certain related parts
# will keep the oracle_id to keep them all connected


#text_df[['type_line']].isna().sum()
#(text_df['type_line'].values == '').sum()

#text_df['super_type']
#text_df['sub_type']


/var/folders/h6/qk346zkn507077y1r9ywwmc80000gn/T/ipykernel_71692/1714090876.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df['super_type'] = text_df['type_line'].apply(lambda x: x.split('—', 1)[0] if '—' in x else x)
/var/folders/h6/qk346zkn507077y1r9ywwmc80000gn/T/ipykernel_71692/1714090876.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df['sub_type'] = text_df['type_line'].apply(lambda x: x.split('—', 1)[1] if '—' in x else x)


0                                      Artifact
1                                          Aura
2                                       Sorcery
3                                          Bird
4                                        Sponge
                          ...                  
30448                                   Instant
30449                           Minotaur Cleric
30450     God // Legendary Artifact — Equipment
30451                                     Samut
30452                                    Sliver
Name: sub_type, Length: 30453, dtype: object

In [5]:
# Start creating BoW model
#text_tk_df = text_df
#text_tk_df['oracle_text'] = text_df['oracle_text'].dropna().apply(word_tokenize)
#text_tk_df
# 154 is apparently the average number of words per card
def generateBOW(textCategory='oracle_text', numberOfWords=154):

    # Turn all the text from one category (pandas column) into a string
    textList = list(text_df[textCategory].dropna().values)
    textListFiltered = [currText.replace('\n', ' ').lower() for currText in textList]
    #print(sum([len(line) for line in textListFiltered])/len(textListFiltered))
    textStr = ' '.join(textListFiltered)
    textTokenized = word_tokenize(textStr)

    # now have a list of every word in every card

    # create a set of all possible words
    possibleWords = list(set(textTokenized))
    totalWords = len(textTokenized)
    # initialize counts of words to be zero
    # then count all of them up
    # (to be used for probabilities)
    counts = {word : 0 for word in possibleWords}
    for word in textTokenized:
        counts[word] += 1
    
    # create probability dictionary where the key is a word
    # and the value is (# instances of the word) / (# all words)
    probs = {word : counts[word]/totalWords for word in possibleWords}
    
    generated = np.random.choice(list(probs.keys()), size=numberOfWords, p=list(probs.values()))
    result = ' '.join(generated)
    return result
# Idea: remove the name from each of them and replace it with the newly generated name for the generated card

In [6]:
def printNicely(input, wordsPerLine):
    ugh = input.split(' ')
    #print(ugh)
    i = 0
    while (i < len(input)/wordsPerLine):
        print(' '.join(ugh[i:i+wordsPerLine]))
        i += wordsPerLine

In [ ]:
generatedName = generateBOW('name', 2)
generatedText = generateBOW('oracle_text', 50)
#test = "the text on this card reads word1 word2 word3 word4 blah blah enchant target creature until its owner pays 3 life (can only be paid as a sorcery) enchanted creature is stupid and silly"
printNicely(generatedName, 2)
printNicely(generatedText, 5)

In [ ]:
text_df = card_df[['oracle_id', 'name', 'type_line', 'keywords', 'oracle_text']]

mana_df = card_df[['color_identity', 'colors', 'mana_cost', 'cmc', 'produced_mana', 'loyalty']]
meta_df = card_df[['legalities', 'layout', 'edhrec_rank', 'penny_rank', 'reserved', 'set_name']]